In [ ]:
from ase.io import read
from ase import Atoms
from ase.optimize import *
from ase.visualize import view
from ase.md import *
from ase.calculators.mopac import *
from ase.constraints import *

import nglview

In [ ]:
#
# In this tutorial notebook, you will explore the physical phenomenon of geometrical frustration
# for sets of oxygen position-constrained, yet otherwise freely rotatable water molecules.
# For now, consider the set of three such water molecules, whose oxygen atoms are arranged in a triangle
#
instem = "3h2o.flat_triangle"
wdg_init = nglview.show_structure_file("input/%s.pdb" % (instem))
wdg_init.add_representation('ball+stick')
wdg_init.center_view()
wdg_init.display(gui=True)

In [ ]:
#
# optimize the geometry of this water set with BFGS
#

# number of optimization iterations
numsteps = 20

calc = Mopac(restart=0, spin=0, OPT=False, functional='PM6', job_type='NOANCI 1SCF GRADIENTS AUX(0,PRECISION=9)', RELSCF=0.0001)

model = "%s" % (instem)
water   = read("input/%s.pdb" % (model), format="pdb")
molecule = Atoms(water)

# add constraints on oxygen atoms
c = FixAtoms(indices=[atom.index for atom in molecule if atom.symbol == 'O'])
molecule.set_constraint(c)
molecule.set_calculator(calc)

print "model", model
ener = molecule.get_potential_energy()
print "potential energy:", ener
grad = molecule.get_forces()
print "gradient", grad

dyn =  QuasiNewton(molecule, trajectory = "output/" + model + '.water.QN_opt.traj')
dyn.run(fmax=0.005, steps = numsteps)
outfile = "output/" + model + ".QN_opt.pdb"
molecule.write(outfile)

In [ ]:
#
# compare the initial structure with ...
#
instem = "3h2o.flat_triangle"
wdg_init = nglview.show_structure_file("input/%s.pdb" % (instem))
wdg_init.add_representation('ball+stick')
wdg_init.center_view()
wdg_init.display(gui=True)

In [ ]:
#
# ... with the optimized structure
#
wdg_final = nglview.show_ase(molecule)
wdg_final.add_representation('ball+stick')
wdg_final.center_view(range(3))
wdg_final.display(gui=True)

In [ ]:
#
# Now delete the third water molecule and reoptimize
#

#
# remember optimized structure visualization from above with three H2O
#
wdg_final_3h2o = nglview.show_ase(molecule)
wdg_final_3h2o.add_representation('ball+stick')
wdg_final_3h2o.center_view(range(3))

calc = Mopac(restart=0, spin=0, OPT=False, functional='PM6', job_type='NOANCI 1SCF GRADIENTS AUX(0,PRECISION=9)', RELSCF=0.0001)

# delete constraints
del molecule.constraints
# delete third water molecule
del molecule[range(6,9)]

# reintroduce constraints on oxygens
c = FixAtoms(indices=[atom.index for atom in molecule if atom.symbol == 'O'])
molecule.set_constraint(c)
molecule.set_calculator(calc)

print "model", model
ener = molecule.get_potential_energy()
print "potential energy:", ener
grad = molecule.get_forces()
print "gradient", grad

dyn = QuasiNewton(molecule, trajectory = "output/" + model + '.2h2o.QN_opt.traj')
dyn.run(fmax=0.005, steps = numsteps)
outfile = "output/" + model + ".QN_opt.2h2o.pdb"
molecule.write(outfile)

In [ ]:
wdg_final_3h2o.display(gui=True)

In [ ]:
wdg_final_2h2o = nglview.show_ase(molecule)
wdg_final_2h2o.add_representation('ball+stick')
wdg_final_2h2o.center_view(range(3))
wdg_final_2h2o.display(gui=True)

In [ ]:
#
# Apparently, the two water molecules alone can adopt a different ground state (G2) compared to 
# their orientation state in G3, with a lower energy.
#
# Reversely, it appears that in the presence of the 3rd water molecule, the two remaining water molecules
# are geometrically frustrated, i.e. their orientation from G2 can no longer contribute to a ground state.
#
# Can you verify this by adding back the third water molecule?
# (
# - copy input/3h2o.flat_triangle.pdb to input/3h2o.flat_triangle.with_G2.pdb
# - in input/3h2o.flat_triangle.with_G2.pdb: replace the coordinates of the first two waters
#   with the ones from output/3h2o.flat_triangle.QN_opt.2h2o.pdb
# )

In [ ]:
#
# Now add the third water molecule back again and reoptimize
#

calc = Mopac(restart=0, spin=0, OPT=False, functional='PM6', job_type='NOANCI 1SCF GRADIENTS AUX(0,PRECISION=9)', RELSCF=0.0001)

model = "3h2o.flat_triangle.with_G2"
water   = read("input/%s.pdb" % (model), format="pdb")
molecule = Atoms(water)

# add constraints on oxygen atoms
c = FixAtoms(indices=[atom.index for atom in molecule if atom.symbol == 'O'])
molecule.set_constraint(c)
molecule.set_calculator(calc)

print "model", model
ener = molecule.get_potential_energy()
print "potential energy:", ener
grad = molecule.get_forces()
print "gradient", grad

dyn = QuasiNewton(molecule, trajectory = "output/" + model + '.water.QN_opt.traj')
dyn.run(fmax=0.005, steps = numsteps)
outfile = "output/" + model + ".QN_opt.pdb"
molecule.write(outfile)

In [ ]:
#
# visualize the optimized structure
#
wdg_final_2h2o = nglview.show_ase(molecule)
wdg_final_2h2o.add_representation('ball+stick')
wdg_final_2h2o.center_view(range(3))
wdg_final_2h2o.display(gui=True)

In [ ]:
# Hä? 
# This is very odd:
# After optimization, the water molecules have now orientated out of the 2D-plane !

# TASK:
# Can you find an explanatation for what has happend?